In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('missdata').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/ContainsNull.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- Id: string (nullable = true)
-- Name: string (nullable = true)
-- Sales: double (nullable = true)

In [0]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Drop the missing data
df.na.drop().show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Used thresh means row contains min two non-value 
df.na.drop(thresh=2).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# all or any in how
'''
    Drop all the null values rows.
'''
df.na.drop(how='any').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Only drop the rows if all the values are null.
df.na.drop(how='all').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Using subset - To drop by specific column
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# By using name where name is null
df.na.drop(subset=['Name']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Filling the missing values

In [0]:
df.printSchema()

root
-- Id: string (nullable = true)
-- Name: string (nullable = true)
-- Sales: double (nullable = true)

In [0]:
# Define subset
df.na.fill('No Name',subset=['Name']).show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|No Name| null|
emp3|No Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [0]:
# For numeric value
df.na.fill(0,subset=['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Mean values

In [0]:
from pyspark.sql.functions import mean

In [0]:
mean_value = df.select(mean(df['Sales'])).collect()

In [0]:
# Display mean value
mean_value

Out[28]: [Row(avg(Sales)=400.5)]

In [0]:
# Display only value
mean_sales = mean_value[0][0]

In [0]:
# Fill the mean value to null in data frame.

df.na.fill(mean_sales,['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# Single line command
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+